In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

URL = "https://www.bluer.co.kr/search?query=&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8"  # 실제 목록 URL로 교체

def clean_text(s: str) -> str:
    return " ".join(s.split()) if s else s

def wait_cards_stable(driver, css, min_wait=6, check_every=0.6):
    """카드 개수가 증가가 멈출 때까지 잠깐 대기"""
    end = time.time() + min_wait
    prev = -1
    while time.time() < end:
        cur = len(driver.find_elements(By.CSS_SELECTOR, css))
        if cur == prev and cur > 0:
            break
        prev = cur
        time.sleep(check_every)
    return prev

opts = Options()
opts.add_argument("--headless=new")        # 안 보이면 주석 처리 후 확인
opts.add_argument("--window-size=1280,2000")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=opts)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator,'webdriver',{get:()=>undefined});"
})

try:
    driver.get(URL)

    CARD_SEL = "li.rl-col.restaurant-thumb-item"

    # 1) 기본 페이지에서 먼저 탐색
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD_SEL))
        )
    except:
        pass

    count = len(driver.find_elements(By.CSS_SELECTOR, CARD_SEL))

    # 2) 카드가 안 보이면 iframe 내부일 수 있으니 프레임 자동 전환 시도
    if count == 0:
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        for f in iframes:
            driver.switch_to.default_content()
            driver.switch_to.frame(f)
            if len(driver.find_elements(By.CSS_SELECTOR, CARD_SEL)) > 0:
                break
        # 프레임 전환 후에도 한 번 더 대기
        if len(driver.find_elements(By.CSS_SELECTOR, CARD_SEL)) == 0:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD_SEL))
            )

    # 3) 개수 안정화까지 잠깐 기다림(동적 템플릿이 한번에 쏟아질 때 대비)
    stable_count = wait_cards_stable(driver, CARD_SEL)
    print(f"[DEBUG] cards: {stable_count}")

    cards = driver.find_elements(By.CSS_SELECTOR, CARD_SEL)
    results = []

    for idx, card in enumerate(cards, 1):
        # (A) 직접 셀렉터로 시도
        name = None
        address = None
        try:
            h3 = card.find_element(By.CSS_SELECTOR, ".header-title h3")
            name = clean_text(h3.text) or clean_text(h3.get_attribute("innerText"))
        except:
            pass
        try:
            addr_el = card.find_element(By.CSS_SELECTOR, ".thumb-caption .info .info-item .content-info.juso-info")
            address = clean_text(addr_el.text) or clean_text(addr_el.get_attribute("innerText"))
        except:
            pass

        # (B) 그래도 못 찾으면 innerHTML을 BS로 파싱해서 재시도(사이트에 따라 .text가 빈 경우가 있음)
        if not name or not address:
            html = card.get_attribute("innerHTML")
            soup = BeautifulSoup(html, "html.parser")
            if not name:
                name_el = soup.select_one(".header-title h3, .clearfix > h3")
                name = clean_text(name_el.get_text()) if name_el else None
            if not address:
                addr_el = soup.select_one(".content-info.juso-info")
                address = clean_text(addr_el.get_text()) if addr_el else None

        if name or address:
            results.append({"name": name, "address": address})

    # 출력
    for r in results:
        print(r)

    if not results:
        print("\n[HINT] 결과가 없으면 다음을 확인하세요:")
        print("- headless 모드 해제하고 실제 카드가 렌더되는지 확인")
        print("- 선택자: 카드(li.rl-col.restaurant-thumb-item) / 이름(.header-title h3) / 주소(.content-info.juso-info)")
        print("- 리스트가 iframe 안에 있는지(코드에 자동 전환 로직 포함)")
        print("- 로그인/지역 선택 등 접근 게이트 유무")

finally:
    driver.quit()


[DEBUG] cards: 32
{'name': '다이닝마', 'address': '서울특별시 강남구 언주로152길 8 (신사동) 유일빌딩 2층'}
{'name': '쵸이닷', 'address': '서울특별시 강남구 도산대로 457 (청담동) 앙스돔빌딩 3층'}
{'name': '비스트로드욘트빌', 'address': '서울특별시 강남구 선릉로158길 13-7 (청담동) 이안빌딩 1층'}
{'name': '밍글스', 'address': '서울특별시 강남구 도산대로67길 19 (청담동) 힐탑빌딩 2층'}
{'name': '라미띠에', 'address': '서울특별시 강남구 도산대로67길 30 (청담동) 2층'}
{'name': '권숙수', 'address': '서울특별시 강남구 압구정로80길 37 (청담동) 이에스빌딩 4층'}
{'name': '레스쁘아', 'address': '서울특별시 강남구 도산대로56길 10 (청담동) 2, 3층'}
{'name': '벽제갈비더청담', 'address': '서울특별시 강남구 도산대로81길 25 (청담동) 조은빌딩 1층'}
{'name': '홍보각', 'address': '서울특별시 강남구 봉은사로 130 (역삼동) 노보텔앰배서더강남서울 LL층'}
{'name': '낙원', 'address': '서울특별시 강서구 방화대로 94 (외발산동) 메이필드호텔 1층'}
{'name': '삼원가든', 'address': '서울특별시 강남구 언주로 835 (신사동)'}
{'name': '강민철레스토랑', 'address': '서울특별시 강남구 도산대로63길 18 (청담동) 청담빌딩 5층'}
{'name': '파씨오네', 'address': '서울특별시 강남구 언주로164길 39 (신사동) 2층'}
{'name': '세븐스도어', 'address': '서울특별시 강남구 학동로97길 41 (청담동) 리유빌딩 4층'}
{'name': '봉래헌', 'address': '서울특별시 강서구 방화대로 94 (외발산동) 메이필드호텔'}
{'name':

In [8]:
!pip install selenium beautifulsoup4 pandas

In [9]:
# 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

URL = "https://www.bluer.co.kr/search?query=&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8"

def clean_text(s: str) -> str:
    return " ".join(s.split()) if s else s

def get_cards_on_page(driver):
    """현재 페이지의 카드에서 필요한 정보 추출"""
    rows = []

    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item"))
    )
    time.sleep(0.6)  # 렌더 안정화

    cards = driver.find_elements(By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item")
    for card in cards:
        name = address = food_type = label = None
        ribbon_count = 0

        # 가게명
        try:
            h3 = card.find_element(By.CSS_SELECTOR, ".header-title h3")
            name = clean_text(h3.text) or clean_text(h3.get_attribute("innerText"))
        except:
            pass

        # 주소
        try:
            addr_el = card.find_element(By.CSS_SELECTOR, ".thumb-caption .info .info-item .content-info.juso-info")
            address = clean_text(addr_el.text) or clean_text(addr_el.get_attribute("innerText"))
        except:
            pass

        # 카테고리(음식 종류)
        try:
            ft = card.find_element(By.CSS_SELECTOR, ".header-status .foodtype li")
            food_type = clean_text(ft.text)
        except:
            pass

        # 리본 개수(이미지 개수)
        try:
            ribbon_imgs = card.find_elements(By.CSS_SELECTOR, ".header-title .ribbons .img-ribbon")
            ribbon_count = len(ribbon_imgs)
        except:
            ribbon_count = 0

        # 라벨(예: '서울 2025 선정') — 여러 개면 공백으로 연결
        try:
            label_els = card.find_elements(By.CSS_SELECTOR, ".header-title .header-labels li")
            labels = [clean_text(el.text) for el in label_els if clean_text(el.text)]
            label = " ".join(labels) if labels else None
        except:
            pass

        # .text가 비는 특수 케이스 대비: innerHTML 보조 파싱
        if not (name and address and (food_type or label or ribbon_count >= 0)):
            html = card.get_attribute("innerHTML")
            soup = BeautifulSoup(html, "html.parser")
            if not name:
                el = soup.select_one(".header-title h3, .clearfix > h3")
                name = clean_text(el.get_text()) if el else name
            if not address:
                el = soup.select_one(".content-info.juso-info")
                address = clean_text(el.get_text()) if el else address
            if not food_type:
                el = soup.select_one(".header-status .foodtype li")
                food_type = clean_text(el.get_text()) if el else food_type
            if ribbon_count == 0:
                ribbon_count = len(soup.select(".header-title .ribbons .img-ribbon"))
            if not label:
                labels = [clean_text(li.get_text()) for li in soup.select(".header-title .header-labels li")]
                label = " ".join(labels) if labels else label

        if name or address:
            rows.append({
                "가게": name,
                "주소": address,
                "카테고리": food_type,
                "리본개수": ribbon_count,
                "라벨": label,
            })

    return rows

def click_next_page(driver):
    """다음 페이지로 이동. 성공시 True, 없으면 False."""
    candidates = [
        (By.CSS_SELECTOR, 'a[rel="next"]'),
        (By.XPATH, "//a[contains(normalize-space(.), '다음')]"),
        (By.CSS_SELECTOR, ".pagination a.next, .paginate a.next, a.page-link.next"),
    ]
    for by, sel in candidates:
        try:
            btns = driver.find_elements(by, sel)
            for b in btns:
                if b.is_displayed() and b.is_enabled():
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", b)
                    time.sleep(0.2)
                    b.click()
                    return True
        except:
            continue
    return False

def crawl_and_save(url=URL, headless=False, delay_between_pages=0.8, out_csv="bluer_gangnam.csv"):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,2200")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=opts)
    try:
        driver.get(url)

        all_rows = []
        page = 1
        while True:
            page_rows = get_cards_on_page(driver)
            print(f"[PAGE {page}] rows: {len(page_rows)}")
            all_rows.extend(page_rows)

            if not click_next_page(driver):
                break
            page += 1
            time.sleep(delay_between_pages)

    finally:
        driver.quit()

    # DataFrame 만들고 컬럼 순서 고정
    df = pd.DataFrame(all_rows, columns=["가게", "주소", "카테고리", "리본개수", "라벨"])
    # 중복 제거(가게+주소 기준)
    df = df.drop_duplicates(subset=["가게", "주소"])

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"saved: {out_csv} ({len(df)} rows)")
    return df

if __name__ == "__main__":
    crawl_and_save(headless=False)  # 처음엔 False로 화면 확인 권장


[PAGE 1] rows: 32
saved: bluer_gangnam.csv (32 rows)


In [ ]:
# pip install selenium beautifulsoup4 pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

URL = "https://www.bluer.co.kr/search?query=&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8#restaurant-filter-bottom"

def clean_text(s: str) -> str:
    return " ".join(s.split()) if s else s

def yesno(flag: bool) -> str:
    return "Y" if flag else "N"

def get_cards_on_page(driver):
    rows = []
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item"))
    )
    time.sleep(0.6)

    cards = driver.find_elements(By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item")
    for card in cards:
        name = address = None
        ribbon_count = 0
        food_types_joined = None
        labels_joined = None
        has_red = False
        has_seoul2025 = False

        # 가게명
        try:
            h3 = card.find_element(By.CSS_SELECTOR, ".header-title h3")
            name = clean_text(h3.text) or clean_text(h3.get_attribute("innerText"))
        except:
            pass

        # 주소
        try:
            addr_el = card.find_element(By.CSS_SELECTOR, ".thumb-caption .info .info-item .content-info.juso-info")
            address = clean_text(addr_el.text) or clean_text(addr_el.get_attribute("innerText"))
        except:
            pass

        # 카테고리(여러 개 → ", "로 연결)
        try:
            ft_els = card.find_elements(By.CSS_SELECTOR, ".header-status .foodtype li")
            food_types = [clean_text(el.text) for el in ft_els if clean_text(el.text)]
            food_types_joined = ", ".join(food_types) if food_types else None
        except:
            pass

        # 리본 개수(이미지 수)
        try:
            ribbon_imgs = card.find_elements(By.CSS_SELECTOR, ".header-title .ribbons .img-ribbon")
            ribbon_count = len(ribbon_imgs)
        except:
            ribbon_count = 0

        # 라벨 수집 + 개별 플래그(레드리본 선정 / 서울 2025 선정)
        try:
            label_els = card.find_elements(By.CSS_SELECTOR, ".header-title .header-labels li")
            labels = [clean_text(el.text) for el in label_els if clean_text(el.text)]
            labels_joined = " ".join(labels) if labels else None
            if labels:
                # 개별 라벨 컬럼 플래그 설정
                has_red = any("레드리본 선정" in t for t in labels)
                has_seoul2025 = any("서울 2025 선정" in t for t in labels)
        except:
            pass

        # 보조 파싱(혹시 text가 비는 케이스)
        if not (name and address):
            html = card.get_attribute("innerHTML")
            soup = BeautifulSoup(html, "html.parser")
            if not name:
                el = soup.select_one(".header-title h3, .clearfix > h3")
                name = clean_text(el.get_text()) if el else name
            if not address:
                el = soup.select_one(".content-info.juso-info")
                address = clean_text(el.get_text()) if el else address
            if food_types_joined is None:
                fts = [clean_text(li.get_text()) for li in soup.select(".header-status .foodtype li") if clean_text(li.get_text())]
                food_types_joined = ", ".join(fts) if fts else None
            if ribbon_count == 0:
                ribbon_count = len(soup.select(".header-title .ribbons .img-ribbon"))
            if labels_joined is None:
                labs = [clean_text(li.get_text()) for li in soup.select(".header-title .header-labels li")]
                labels_joined = " ".join(labs) if labs else None
                has_red = any("레드리본 선정" in t for t in labs)
                has_seoul2025 = any("서울 2025 선정" in t for t in labs)

        if name or address:
            rows.append({
                "가게": name,
                "주소": address,
                "카테고리": food_types_joined,
                "리본개수": ribbon_count,
                "레드리본 선정": yesno(has_red),
                "서울 2025 선정": yesno(has_seoul2025),
                "라벨": labels_joined,  # 원문 라벨도 유지(원하면 제거 가능)
            })
    return rows

def click_next_page(driver):
    candidates = [
        (By.CSS_SELECTOR, 'a[rel="next"]'),
        (By.XPATH, "//a[contains(normalize-space(.), '다음')]"),
        (By.CSS_SELECTOR, ".pagination a.next, .paginate a.next, a.page-link.next"),
    ]
    for by, sel in candidates:
        try:
            btns = driver.find_elements(by, sel)
            for b in btns:
                if b.is_displayed() and b.is_enabled():
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", b)
                    time.sleep(0.2)
                    b.click()
                    return True
        except:
            continue
    return False

def crawl_and_save(url=URL, headless=False, delay_between_pages=0.8, out_csv="bluer_gangnam".csv"):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,2200")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=opts)
    try:
        driver.get(url)

        all_rows = []
        page = 1
        while True:
            page_rows = get_cards_on_page(driver)
            print(f"[PAGE {page}] rows: {len(page_rows)}")
            all_rows.extend(page_rows)

            if not click_next_page(driver):
                break
            page += 1
            time.sleep(delay_between_pages)
    finally:
        driver.quit()

    # 원하는 컬럼 순서로 저장
    cols = ["가게", "주소", "카테고리", "리본개수", "레드리본 선정", "서울 2025 선정", "라벨"]
    df = pd.DataFrame(all_rows)
    df = df[cols]
    df = df.drop_duplicates(subset=["가게", "주소"])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"saved: {out_csv} ({len(df)} rows)")
    return df

if __name__ == "__main__":
    crawl_and_save(headless=False)


[PAGE 1] rows: 32
saved: bluer_gangnam.csv (32 rows)


## 페이지 넘어가는 코드

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

START_URL = "https://www.bluer.co.kr/search?query=&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%B6%81"

def clean_text(s: str) -> str:
    return " ".join(s.split()) if s else s

def yesno(flag: bool) -> str:
    return "Y" if flag else "N"

def wait_cards(driver, timeout=15):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item"))
    )
    time.sleep(0.4)  # 렌더 안정화

def get_total_pages(driver) -> int:
    """ul.pagination.bootpag에서 마지막 페이지(data-lp) 추출"""
    try:
        pager = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.pagination.bootpag"))
        )
    except:
        return 1
    # last 버튼 우선
    last = pager.find_elements(By.CSS_SELECTOR, "li.last[data-lp]")
    if last:
        try:
            return int(last[0].get_attribute("data-lp"))
        except:
            pass
    # 보조: 숫자 li 중 최대 data-lp
    nums = pager.find_elements(By.CSS_SELECTOR, "li[data-lp]")
    max_lp = 1
    for li in nums:
        try:
            lp = int(li.get_attribute("data-lp"))
            if lp > max_lp:
                max_lp = lp
        except:
            continue
    return max_lp

def go_to_page(driver, page: int, timeout=12) -> bool:
    """li[data-lp=page] 클릭 후 active 페이지/카드 갱신 대기"""
    # 현재 첫 카드(있으면) 참조해서 staleness 체크
    old_first = None
    try:
        old_first = driver.find_element(By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item")
    except:
        pass

    # 페이지네이션 보이도록 살짝 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.2)

    # 타겟 li[data-lp="page"] > a 클릭
    btn = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f'ul.pagination.bootpag li[data-lp="{page}"] a'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    time.sleep(0.1)
    btn.click()

    # 1) active 페이지가 page로 바뀔 때까지 대기
    WebDriverWait(driver, timeout).until(
        lambda d: (d.find_element(By.CSS_SELECTOR, "ul.pagination.bootpag li.active")
                   .get_attribute("data-lp") == str(page))
    )
    # 2) 카드가 새로고침(staleness) 혹은 다시 로드될 때까지 대기
    if old_first:
        try:
            WebDriverWait(driver, 6).until(EC.staleness_of(old_first))
        except:
            pass
    wait_cards(driver, timeout=timeout)
    return True

def extract_cards(driver):
    """현재 페이지 카드 → dict 리스트"""
    rows = []
    cards = driver.find_elements(By.CSS_SELECTOR, "li.rl-col.restaurant-thumb-item")
    for card in cards:
        name = address = None
        food_types_joined = None
        ribbon_count = 0
        labels_joined = None
        has_red = False
        has_seoul2025 = False

        # 가게명
        try:
            h3 = card.find_element(By.CSS_SELECTOR, ".header-title h3")
            name = clean_text(h3.text) or clean_text(h3.get_attribute("innerText"))
        except: pass
        # 주소
        try:
            addr_el = card.find_element(By.CSS_SELECTOR, ".thumb-caption .info .info-item .content-info.juso-info")
            address = clean_text(addr_el.text) or clean_text(addr_el.get_attribute("innerText"))
        except: pass
        # 카테고리(여러 개)
        try:
            ft_els = card.find_elements(By.CSS_SELECTOR, ".header-status .foodtype li")
            food_types = [clean_text(el.text) for el in ft_els if clean_text(el.text)]
            food_types_joined = ", ".join(food_types) if food_types else None
        except: pass
        # 리본 개수(이미지 개수)
        try:
            ribbon_count = len(card.find_elements(By.CSS_SELECTOR, ".header-title .ribbons .img-ribbon"))
        except: ribbon_count = 0
        # 라벨 + 플래그
        try:
            label_els = card.find_elements(By.CSS_SELECTOR, ".header-title .header-labels li")
            labels = [clean_text(el.text) for el in label_els if clean_text(el.text)]
            labels_joined = " ".join(labels) if labels else None
            has_red = any("레드리본 선정" in t for t in labels)
            has_seoul2025 = any("서울 2025 선정" in t for t in labels)
        except: pass

        # 보조 파싱
        if not (name and address):
            html = card.get_attribute("innerHTML")
            soup = BeautifulSoup(html, "html.parser")
            if not name:
                el = soup.select_one(".header-title h3, .clearfix > h3")
                name = clean_text(el.get_text()) if el else name
            if not address:
                el = soup.select_one(".content-info.juso-info")
                address = clean_text(el.get_text()) if el else address
            if food_types_joined is None:
                fts = [clean_text(li.get_text()) for li in soup.select(".header-status .foodtype li") if clean_text(li.get_text())]
                food_types_joined = ", ".join(fts) if fts else None
            if ribbon_count == 0:
                ribbon_count = len(soup.select(".header-title .ribbons .img-ribbon"))
            if labels_joined is None:
                labs = [clean_text(li.get_text()) for li in soup.select(".header-title .header-labels li")]
                labels_joined = " ".join(labs) if labs else None
                has_red = any("레드리본 선정" in t for t in labs)
                has_seoul2025 = any("서울 2025 선정" in t for t in labs)

        if name or address:
            rows.append({
                "가게": name,
                "주소": address,
                "카테고리": food_types_joined,
                "리본개수": ribbon_count,
                "레드리본 선정": yesno(has_red),
                "서울 2025 선정": yesno(has_seoul2025),
                "라벨": labels_joined,
            })
    return rows

def crawl_all_pages_click(start_url=START_URL, headless=False, out_csv="bluer_gangnam.csv"):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,2200")
    # (필요 시 탐지 회피 옵션 추가 가능)
    driver = webdriver.Chrome(options=opts)

    all_rows, seen = [], set()
    try:
        driver.get(start_url)
        wait_cards(driver)

        total = get_total_pages(driver)
        print(f"[INFO] total pages: {total}")

        # 현재 활성 페이지
        try:
            cur = int(driver.find_element(By.CSS_SELECTOR, "ul.pagination.bootpag li.active").get_attribute("data-lp"))
        except:
            cur = 1

        # cur ~ total까지 순회
        for page in range(cur, total + 1):
            if page != cur:
                go_to_page(driver, page)

            page_rows = extract_cards(driver)
            print(f"[PAGE {page}] rows: {len(page_rows)}")

            for r in page_rows:
                key = (r["가게"], r["주소"])
                if key in seen:
                    continue
                seen.add(key)
                all_rows.append(r)

    finally:
        driver.quit()

    df = pd.DataFrame(all_rows, columns=["가게", "주소", "카테고리", "리본개수", "레드리본 선정", "서울 2025 선정", "라벨"])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"saved: {out_csv} ({len(df)} rows)")
    return df

if __name__ == "__main__":
    crawl_all_pages_click(headless=False)


[INFO] total pages: 130


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=141.0.7390.77)
Stacktrace:
	GetHandleVerifier [0x0x7ff7109be9e5+80021]
	GetHandleVerifier [0x0x7ff7109bea40+80112]
	(No symbol) [0x0x7ff71074060f]
	(No symbol) [0x0x7ff7107182f1]
	(No symbol) [0x0x7ff7107c88be]
	(No symbol) [0x0x7ff7107e8fa2]
	(No symbol) [0x0x7ff7107c1003]
	(No symbol) [0x0x7ff7107895d1]
	(No symbol) [0x0x7ff71078a3f3]
	GetHandleVerifier [0x0x7ff710c7dd8d+2960445]
	GetHandleVerifier [0x0x7ff710c7804a+2936570]
	GetHandleVerifier [0x0x7ff710c98a87+3070263]
	GetHandleVerifier [0x0x7ff7109d84ce+185214]
	GetHandleVerifier [0x0x7ff7109dff1f+216527]
	GetHandleVerifier [0x0x7ff7109c7c24+117460]
	GetHandleVerifier [0x0x7ff7109c7ddf+117903]
	GetHandleVerifier [0x0x7ff7109adcb8+11112]
	BaseThreadInitThunk [0x0x7ffeb9e5e8d7+23]
	RtlUserThreadStart [0x0x7ffebae28d9c+44]


In [ ]:
import pandas as pd

# 원본이 utf-8-sig로 저장되었다고 가정 (다르면 encoding 바꿔보세요: 'utf-8', 'cp949' 등)
df = pd.read_csv("blue_food.csv", encoding="utf-8-sig")

# 엑셀 호환 좋게 cp949로 다시 저장
df.to_csv("bluer_food_cp949.csv", index=False, encoding="cp949")

# 또는 UTF-8 with BOM으로 다시 저장
df.to_csv("bluer_food_utf8sig.csv", index=False, encoding="utf-8-sig")
